In [1]:
import sframe

In [2]:
csv=sframe.SFrame.read_csv("/Users/singhv/Documents/Curation/thesaurus/owl/ThesaurusSemTypePrefName.tsv",delimiter="\t")

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging /tmp/sframe_server_1474077815.log


------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/singhv/Documents/Curation/thesaurus/owl/ThesaurusSemTypePrefName.tsv

Parsing completed. Parsed 120280 lines in 0.155027 secs.

In [3]:
csv.head(5)

cui,name,type
C55602,CTCAE Grade 1 BreastFunction and Lactation ...,Finding
C55603,CTCAE Grade 2 BreastFunction and Lactation ...,Finding
C55624,CTCAE Grade 3 Infertilityand Sterility ...,Finding
C116921,Cognitive Impairment,Finding
C55605,CTCAE Grade 1 BreastNipple and Areolar ...,Finding


In [36]:
csv.filter_by(["Ovarian Infection", "Fucosidosis"],"name")

cui,name,type
C61274,Fucosidosis,Disease or Syndrome
C78512,Ovarian Infection,Disease or Syndrome


In [11]:
from sframe import SArray
STOP_WORDS = ["a", "an", "and", "are", "as", "at", "be", "but", "by",
"for", "if", "in", "into", "is", "it",
"no", "not", "of", "on", "or", "such",
"that", "the", "their", "then", "there", "these",
"they", "this", "to", "was", "will", "with"]
names = csv.filter_by(["Disease or Syndrome"],"type")["name"].apply(lambda x: x.lower())
lengths = names.apply(lambda name: len([ word for word in name.split() if word not in STOP_WORDS ]))

all_lengths = csv["name"].apply(lambda name: len([ word for word in name.split()]))

In [23]:
#print names[2]
#print lengths[2]
single_word_concepts = SArray.where(all_lengths == 1, csv["name"], None).apply(lambda x: x.lower()).unique().dropna()
single_word_diseases = SArray.where(lengths == 1, names, None).unique().dropna()
two_word_diseases = SArray.where(lengths == 2, names, None).unique().dropna()
print len(single_word_diseases)
print len(two_word_diseases)
print single_word_concepts[200]
single_word_concepts.sort().save("/Users/singhv/Desktop/single_word_concepts.txt")
#single_word_diseases.sort().save("/Users/singhv/Desktop/single_word_diseases.txt")

590
1771
dermomyotome


In [9]:
d1 = single_word_diseases[200]
d2 = two_word_diseases[101]
print [d1, d2]

['megacolon', 'cerebral sarcoidosis']


In [17]:
first_words = two_word_diseases.apply(lambda d: d.split()[0]).sort()
second_words = two_word_diseases.apply(lambda d: d.split()[1]).sort()

common = [word for word in first_words if word in single_word_concepts]
for word in common:
    print word+"\n"